# Filtros - introdução



Bem vindos ao módulo de filtros! Neste material estudaremos melhor como filtros são aplicados no mundo musical de forma interativa. Todo o material é construído em Python utilizando o Jupyter Notebook. Sinta-se livre para trazer seu áudio, **apenas garantindo que este áudio esteja no formato mono e ele esteja na pasta audioFiles** (um áudio mono se refere a um tipo de áudio que é reproduzido através de um único canal de áudio. Isso significa que o som é reproduzido de maneira unidimensional, sem separação espacial entre os canais de áudio esquerdo e direito)

Vamos começar importando as bibliotecas que utilizaremos nesse módulo. Iremos utilizar as seguintes bibliotecas:

- **NumPy**: oferece uma ampla gama de funções e operações de biblioteca que facilitam aos desenvolvedores a execução de cálculos matemáticos e científicos de maneira eficiente. Dentro deste módulo, o NumPy será utilizado para facilitar cálculos entre vetores e matrizes, bem como a geração de vetores nulos e o fácil cálculo de número absolutos.
- **SciPy**: fornece algoritmos para otimização, integração, interpolação, problemas de autovalor, equações algébricas, equações diferenciais e estatísticas e muito mais. Nestre módulo, o SciPy será utlizado para aplicação de transformada de fourier discreta, para calcular a resposta em frequência de um filtro digital, para filtrar dados ao longo de uma dimensão com um filtro IIR ou FIR e para projetar um filtro Butterworth.
- **IPython**: fornece um conjunto de ferramentas abrangente para ajudá-lo a aproveitar ao máximo a utilização interativa do Python (PÉREZ; GRANGER,
2007). Aqui usaremos IPython principalmente para a reprodução do áudio original e do áudio modificaod pelos filtros.
- **Matplotlib**: é uma biblioteca abrangente para criar visualizações estáticas, animadas e interativas em Python. Neste trabalho, serão utilizadas as funções para plotagem dos gráficos relacionados ao filtro, ao áudio de entrada e ao áudio de sáida.
- **Ipywidgets**: ou Jupyter Widgets são widgets HTML interativos para notebooks Jupyter e o kernel IPython (IPYWIDGETS. . . , 2023). Não entraremos em muitos detalhes sobre sua implementação, mas todo código está disponível no arquivo  `widgets.ipynb `

In [21]:
import numpy as np
from IPython.display import Audio, display
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import freqz
from scipy.io import wavfile
import scipy as scipy
import warnings
warnings.simplefilter("ignore", category=wavfile.WavFileWarning)
%run -i widgets.ipynb






Agora, vamos definir uma função que irá abrir o arquivo do nosso áudio e transformar ele em um sinal convertendo os dados para um array através da função $numpy$. Além disso, coletamos a taxa de amostragem do áudio.

In [22]:
def read_audio_file(path):
    # leitura do arquivo WAV
    sample_rate, original_signal = wavfile.read('../audioFiles/' + path)
    return sample_rate, original_signal

Definiremos também uma função que plotará nossos sinais (de entrada e de saída) no domínio da frequência e no domíninio do tempo.

Junto a ela, também definiremos uma função para plotar a resposta em frequência do nosso filtro. Logo em seguida, uma função para mostrar em um player os áudios de entrada e saída. As funções `@out.capture()` são utilizadas pelo widget para capturar suas repostas e apresentar nos dashboards.

In [23]:
@out1.capture()
def plot_signal(data, sample_rate, title):
    # domínio do tempo
    cm = 1/2.54  # centimeters in inches
    plt.figure(figsize=(30*cm, 10*cm))
    plt.suptitle(title)

    time = np.arange(0, len(data)) / (1 * sample_rate)
    plt.subplot(1, 2, 1)
    plt.plot(time, data)
    plt.xlabel("Tempo (s)")
    plt.ylabel("Amplitude")
    plt.title("Sinal no domínio do tempo")

    # domínio da frequência
    plt.subplot(1, 2, 2)
    n = len(data)
    yf = fft(data)
    xf = fftfreq(n, 1 / sample_rate)[: n // 2]
    plt.plot(xf, 2.0 / n * np.abs(yf[0 : n // 2]))
    plt.xlabel("Frequência (Hz)")
    plt.ylabel("Amplitude")
    plt.title("Sinal no domínio da frequência")

    plt.tight_layout()
    plt.show()



In [24]:

@out2.capture()
def plot_filter(cutoff_frequency, sample_rate, w, h):
    # plotagem da resposta em frequência
    %matplotlib inline
    cm = 1/2.54  # centimeters in inches
    plt.figure(figsize=(20*cm, 10*cm))
    plt.plot(sample_rate * w / (2 * np.pi), np.abs(h), "b")
    plt.axvline(cutoff_frequency, color="r")
    plt.xlabel("Frequência (Hz)")
    plt.ylabel("Amplitude")
    plt.title("Resposta em frequência do filtro")
    plt.grid(True)
    plt.show()
    
@out3.capture()
def display_audio(file_name, output_file_name):
    print('Áudio original:')
    display(Audio('../audioFiles/' + file_name))
    print('Áudio alterado:')
    display(Audio('../audioFiles/' + output_file_name)) 


Por fim, definiremos uma função para transformar nosso sinal de saída do filtro em um arquivo wav de áudio.

In [25]:
def write_audio_file(output_signal, sampwidth, name):
    filtered_signal = np.int16(output_signal / np.max(np.abs(output_signal)) * 32767)
    wavfile.write('../audioFiles/' + name, sampwidth, filtered_signal)